## Story Generator Local Server

[Google Notebook](https://colab.research.google.com/drive/1uNygzDR4hISwLOgmDS31hRHfr6KAF7Ib?usp=sharing#scrollTo=unique_id_here)

In [4]:
%pip install -q transformers==4.35.2 accelerate bitsandbytes==0.41.1 huggingface_hub google-generativeai==0.3.1
%pip install -q Flask==2.3.2 Pillow==9.5.0 requests==2.30.0 flask-ngrok Flask-CORS==3.0.10 pyngrok
%pip install Flask-Cors
import os

# Use environment variables
from huggingface_hub import login
from dotenv import load_dotenv
load_dotenv()
login(os.getenv('HUGGINGFACE_API_KEY'))

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from IPython.display import clear_output
clear_output(wait=False)

print("Dependencies installed successfully!")

Dependencies installed successfully!


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import base64
from PIL import Image
import io
import google.generativeai as genai
import os

# Configure Gemini API
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
generation_config = {
    "temperature": 0.7,
    "top_p": 0.8,
    "top_k": 40,
    "max_output_tokens": 8192,
}

# Use gemini-1.5-pro for vision
vision_model = genai.GenerativeModel('gemini-1.5-pro', generation_config=generation_config)

# Use gemini-pro for story generation
story_model = genai.GenerativeModel('gemini-pro', generation_config=generation_config)

# Configure ngrok with auth token from environment
ngrok.set_auth_token(os.getenv('NGROK_AUTH_TOKEN'))

app = Flask(__name__)
CORS(app)

@app.route('/generate_story', methods=['POST'])
def generate_story():
    try:
        data = request.json
        if not data:
            print("DEBUG: No data received in request")
            return jsonify({'success': False, 'error': 'No data received'}), 400

        image_data = data.get('image', '')
        genre = data.get('genre', 'fantasy').lower()
        word_count = data.get('length', 200)

        if 'base64,' in image_data:
            image_data = image_data.split('base64,')[1]

        if not image_data:
            return jsonify({'success': False, 'error': 'No image data provided'}), 400

        try:
            # Process image
            image_bytes = base64.b64decode(image_data)
            image = Image.open(io.BytesIO(image_bytes))
            print(f"DEBUG: Image mode: {image.mode}")
                
            # Convert to RGB if needed
            if image.mode in ('RGBA', 'LA'):
                background = Image.new('RGB', image.size, (255, 255, 255))
                background.paste(image, mask=image.split()[-1])
                image = background
            elif image.mode != 'RGB':
                image = image.convert('RGB')
                
            # Create model for image analysis
            vision_model = genai.GenerativeModel('gemini-1.5-pro-vision')
            
            # Generate image prompt
            image_prompt = "Describe what you see in this image concisely."
            image_response = vision_model.generate_content([image_prompt, image])
            image_description = image_response.text
            print(f"DEBUG: Image description: {image_description}")

        except Exception as img_error:
            print(f"DEBUG: Image processing error: {str(img_error)}")
            return jsonify({'success': False, 'error': f'Image processing error: {str(img_error)}'}), 400

        # Craft prompt
        prompt = f"""Write a {genre} story appropriate for all ages. Base it on this scene: {image_description}
        Requirements:
        - Length: approximately {word_count} words
        - Age-appropriate content
        - Clear narrative structure
        - Engaging and descriptive language
        - Positive message or moral"""

        print(f"DEBUG: Generated prompt: {prompt}")

        # Generate story
        response = model.generate_content(prompt)

        if response.text:
            return jsonify({'success': True, 'story': response.text})
        else:
            return jsonify({
                'success': False,
                'error': 'Story generation failed. Please try again.'
            }), 500

    except Exception as e:
        print(f"DEBUG: Server error: {str(e)}")
        return jsonify({
            'success': False,
            'error': f'Server error: {str(e)}'
        }), 500

print("Starting server...")
try:
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f'\nNgrok tunnel established at: {public_url}')
    print(f'Story generation endpoint: {public_url}/generate_story\n')
    
    # Start Flask app
    app.run(port=5000)
except Exception as e:
    print(f"Error starting server: {str(e)}")


t=2024-11-15T16:32:03+0530 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\Shravan\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\Shravan\\.ngrok2\\ngrok.yml


Starting server...

Ngrok tunnel established at: NgrokTunnel: "https://c271-2409-40f3-101b-f886-7095-d6fa-4e8b-6f3e.ngrok-free.app" -> "http://localhost:5000"
Story generation endpoint: NgrokTunnel: "https://c271-2409-40f3-101b-f886-7095-d6fa-4e8b-6f3e.ngrok-free.app" -> "http://localhost:5000"/generate_story

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


DEBUG: Image mode: RGB


127.0.0.1 - - [15/Nov/2024 16:34:37] "POST /generate_story HTTP/1.1" 400 -


DEBUG: Image processing error: 404 Gemini 1.0 Pro Vision has been deprecated on July 12, 2024. Consider switching to different model, for example gemini-1.5-flash.
